In [12]:
import import_ipynb

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utilities.UtilityFunctions import retrieve_reviews_df, preprocessing, vectorize, text_preprocess

In [10]:
# retrieve and preprocesses dataframe
df = preprocessing(retrieve_reviews_df())

Concatenating ../data/Reviews-4.csv
Concatenating ../data/Reviews-1.csv
Concatenating ../data/Reviews-2.csv
Concatenating ../data/Reviews-3.csv


In [13]:
# perform text preprocessing
text_preprocess(df, ['html', 'punc', 'lower'])

In [14]:
df.head()

,Id,Score,Summary,Text,Sentiment,Clean_text
0,566663,5,THE BEST TREAT YOU CAN BUY PERIOD!,There is NOTHING on the market like these trea...,1,there is nothing on the market like these trea...
1,566664,5,Awesome treats!,My pup can't have anything with grains in it a...,1,my pup cant have anything with grains in it an...
2,566665,4,"My dog loves this, but beware there are differ...",My elderly dog who has pretty much given up on...,0,my elderly dog who has pretty much given up on...
3,566666,5,Awesome treats!,So glad we found Amazon. Used to travel to the...,1,so glad we found amazon used to travel to the ...
4,566667,3,Good stuff,"Naturally, pooch scarfed this up. What's not ...",0,naturally pooch scarfed this up whats not to ...


In [15]:
X = df['Clean_text']
y = df['Sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

In [19]:
X_train_counts, X_train_tfidf = vectorize(X_train)


In [20]:
from sklearn.svm import LinearSVC
clf_linearSVC_pipe = Pipeline([("vect", CountVectorizer()), ("tfidf", TfidfTransformer()), ("clf_linearSVC", LinearSVC(verbose=True))])

In [21]:
clf_linearSVC_pipe.fit(X_train, y_train)

[LibLinear].....*
optimization finished, #iter = 54
Objective value = -74058.492829
nSV = 167347
.....*
optimization finished, #iter = 55
Objective value = -179351.391130
nSV = 344430
.....*
optimization finished, #iter = 54
Objective value = -174194.620312
nSV = 329583


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf_linearSVC', LinearSVC(verbose=True))])

In [22]:
predictedSVM = clf_linearSVC_pipe.predict(X_test)
np.mean(predictedSVM == y_test)

0.8210157803071618